# CS5760 - Homework 1
---
Student: Sai Siva Shankara Vara Prasad Kopparthi
ID: 700765221


## Q1. Regex
Write regex patterns and test them.

In [21]:
import re

text = """ Our office ZIPs include 54321, 54321-9876, and 54321 9876.
Some words: London, don’t, mother-in-law, orange, zebra.
Numbers appear as: -9,876.54, +321, 0.99, 2.5e+8, 6E-3
Spellings to check: email, e-mail, E Mail, e–mail, EMAIL
People shout: go, gooo, gooooo! go, go?
Is this really correct? ”)
"""

# 1. U.S. postal codes
pattern_zip = r"\b\d{5}(?:[-\s]\d{4})?\b"
print("Zip codes found:", re.findall(pattern_zip, text))

# 2. Words starting with lowercase
pattern_lowercase = r"\b(?![A-Z])[A-Za-z’\'-]+\b"
print("Lowercase-initial words:", re.findall(pattern_lowercase, text))

# 3. Different number formats
pattern_number = r"[+-]?\d{1,3}(?:,\d{3})*(?:\.\d+)?(?:[eE][+-]?\d+)?"
print("Numbers detected:", re.findall(pattern_number, text))

# 4. Variations of 'email'
pattern_email = r"(?i)e[ \-–]?mail"
print("Email spellings:", re.findall(pattern_email, text))

# 5. Repeated 'go' with punctuation
pattern_go = r"\bgo+[!\.,\?]?\b"
print("Go-like words:", re.findall(pattern_go, text))

# 6. Questions at line ends
pattern_question = r"\?['\")\]\s]*$"
print("Question-style endings:", re.findall(pattern_question, text, re.MULTILINE))

Zip codes found: ['54321', '54321-9876', '54321 9876']
Lowercase-initial words: ['office', 'include', '-', 'and', 'words', 'don’t', 'mother-in-law', 'orange', 'zebra', 'appear', 'as', '-', 'to', 'check', 'email', 'e-mail', 'e', 'mail', 'shout', 'go', 'gooo', 'gooooo', 'go', 'go', 'this', 'really', 'correct']
Numbers detected: ['543', '21', '543', '21', '-987', '6', '543', '21', '987', '6', '-9,876.54', '+321', '0.99', '2.5e+8', '6E-3']
Email spellings: ['email', 'e-mail', 'E Mail', 'e–mail', 'EMAIL']
Go-like words: ['go', 'gooo', 'gooooo', 'go', 'go']
Question-style endings: ['?']


## Q2. Tokenization
Tokenize a paragraph manually and with a tool, compare results, identify MWEs.

**Example Telugu Paragraph:**

నేను హైదరాబాద్ లో చదువుతున్నాను. 

నా స్నేహితుడు తెలుగు సినిమాలు చూడడం ఇష్టపడతాడు. 

మేము ప్రతి వారాంతం కలిసి బయటికి వెళ్తాము.


**Translation (for your understanding):**

I am studying in Hyderabad.

My friend likes watching Telugu movies.

We go out together every weekend.

In [22]:
import nltk
from nltk.tokenize import word_tokenize

# Telugu paragraph
paragraph = "నేను హైదరాబాద్ లో చదువుతున్నాను. నా స్నేహితుడు తెలుగు సినిమాలు చూడడం ఇష్టపడతాడు. మేము ప్రతి వారాంతం కలిసి బయటికి వెళ్తాము."

# 1. Naive space-based tokenization
space_tokens = paragraph.split(" ")
print("Space-based tokens:", space_tokens)

# 2. Manual correction (handling punctuation)
manual_tokens = [word.strip(".,!?") for word in space_tokens]
print("Manually cleaned tokens:", manual_tokens)

# 3. NLTK tokenizer (it works for Unicode text too)
nltk.download("punkt")
tool_tokens = word_tokenize(paragraph)
print("NLTK tokens:", tool_tokens)

# 4. Multiword Expressions (MWEs in Telugu)
mwes = ["తెలుగు సినిమాలు", "ప్రతి వారాంతం", "హైదరాబాద్ లో"]
print("Multiword Expressions:", mwes)

Space-based tokens: ['నేను', 'హైదరాబాద్', 'లో', 'చదువుతున్నాను.', 'నా', 'స్నేహితుడు', 'తెలుగు', 'సినిమాలు', 'చూడడం', 'ఇష్టపడతాడు.', 'మేము', 'ప్రతి', 'వారాంతం', 'కలిసి', 'బయటికి', 'వెళ్తాము.']
Manually cleaned tokens: ['నేను', 'హైదరాబాద్', 'లో', 'చదువుతున్నాను', 'నా', 'స్నేహితుడు', 'తెలుగు', 'సినిమాలు', 'చూడడం', 'ఇష్టపడతాడు', 'మేము', 'ప్రతి', 'వారాంతం', 'కలిసి', 'బయటికి', 'వెళ్తాము']
NLTK tokens: ['నేను', 'హైదరాబాద్', 'లో', 'చదువుతున్నాను', '.', 'నా', 'స్నేహితుడు', 'తెలుగు', 'సినిమాలు', 'చూడడం', 'ఇష్టపడతాడు', '.', 'మేము', 'ప్రతి', 'వారాంతం', 'కలిసి', 'బయటికి', 'వెళ్తాము', '.']
Multiword Expressions: ['తెలుగు సినిమాలు', 'ప్రతి వారాంతం', 'హైదరాబాద్ లో']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sivaprasadkopparthi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### **Q2. Reflection:**
 1. Tokenization in Telugu is harder than English because of compound words and rich morphology.
 2. Naïve splitting by spaces often leaves punctuation attached to words.
 3. Manual correction improves results, but still misses some cases.
 4. Tools like NLTK handle Unicode but may not fully capture Telugu MWEs, so extra care is needed.

## Q3. Byte Pair Encoding (BPE)
Perform manual merges, implement mini-BPE, train on paragraph.

In [23]:
from collections import defaultdict, Counter
import re

# Build vocab with end-of-word marker
def get_vocab_from_corpus(words):
    vocab = Counter()
    for w in words:
        vocab[' '.join(list(w) + ['_'])] += 1
    return vocab

def get_stats(vocab):
    pairs = Counter()
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, vocab):
    bigram = ' '.join(pair)
    pattern = re.compile(r'(?<!\S)'+re.escape(bigram)+r'(?!\S)')
    new_vocab = {}
    for word, freq in vocab.items():
        new_word = pattern.sub(''.join(pair), word)
        new_vocab[new_word] = freq
    return new_vocab

def bpe_learn(vocab, num_merges):
    merges = []
    v = dict(vocab)
    for i in range(num_merges):
        pairs = get_stats(v)
        if not pairs:
            break
        best = max(pairs.items(), key=lambda x: x[1])[0]
        merges.append(best)
        v = merge_vocab(best, v)
        print(f"Merge {i+1}: {best}")
    return merges

def apply_bpe_to_word(word, merges):
    symbols = list(word) + ['_']
    for a,b in merges:
        i = 0
        new = []
        while i < len(symbols):
            if i < len(symbols)-1 and symbols[i] == a and symbols[i+1] == b:
                new.append(a+b)
                i += 2
            else:
                new.append(symbols[i])
                i += 1
        symbols = new
    return symbols

# 3.1 Manual merges on toy corpus
toy_corpus = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new"
toy_vocab = get_vocab_from_corpus(toy_corpus.split())
print("Initial toy vocab:", toy_vocab)

# First 3 merges manually
v = toy_vocab
for step in range(3):
    pairs = get_stats(v)
    best = max(pairs.items(), key=lambda x:x[1])[0]
    print(f"Step {step+1} merge: {best}")
    v = merge_vocab(best, v)
    print("Updated vocab sample:", list(v.keys())[:5])

# 3.2 Mini-BPE learner on toy corpus
merges = bpe_learn(toy_vocab, 10)
for w in ["new", "newer", "lowest", "wider", "newestest"]:
    print(w, "->", apply_bpe_to_word(w, merges))

# 3.3 BPE on a paragraph
paragraph = "Natural language processing studies how to teach computers to understand and generate human language. Tokenization and subword segmentation are important preprocessing steps."
para_vocab = get_vocab_from_corpus(re.findall(r"\w+", paragraph.lower()))
para_merges = bpe_learn(para_vocab, 30)

for w in ["processing", "tokenization", "subword", "morphological", "coverage"]:
    print(w, "->", apply_bpe_to_word(w, para_merges))


Initial toy vocab: Counter({'n e w e r _': 6, 'l o w _': 5, 'w i d e r _': 3, 'l o w e s t _': 2, 'n e w _': 2})
Step 1 merge: ('e', 'r')
Updated vocab sample: ['l o w _', 'l o w e s t _', 'n e w er _', 'w i d er _', 'n e w _']
Step 2 merge: ('er', '_')
Updated vocab sample: ['l o w _', 'l o w e s t _', 'n e w er_', 'w i d er_', 'n e w _']
Step 3 merge: ('n', 'e')
Updated vocab sample: ['l o w _', 'l o w e s t _', 'ne w er_', 'w i d er_', 'ne w _']
Merge 1: ('e', 'r')
Merge 2: ('er', '_')
Merge 3: ('n', 'e')
Merge 4: ('ne', 'w')
Merge 5: ('l', 'o')
Merge 6: ('lo', 'w')
Merge 7: ('new', 'er_')
Merge 8: ('low', '_')
Merge 9: ('w', 'i')
Merge 10: ('wi', 'd')
new -> ['new', '_']
newer -> ['newer_']
lowest -> ['low', 'e', 's', 't', '_']
wider -> ['wid', 'er_']
newestest -> ['new', 'e', 's', 't', 'e', 's', 't', '_']
Merge 1: ('a', 'n')
Merge 2: ('a', 't')
Merge 3: ('e', '_')
Merge 4: ('d', '_')
Merge 5: ('p', 'r')
Merge 6: ('e', 's')
Merge 7: ('s', 't')
Merge 8: ('t', 'o')
Merge 9: ('e', 'r'

**Reflection:** 
1. BPE learned frequent prefixes (e.g., “sub”), suffixes (e.g., “age_”), and stems (e.g., “wor”).
  2. This shows that BPE often captures morpheme-like units in real text.
  3. A key advantage is that it reduces vocabulary size while handling rare or derived words.
  4. It also generalizes better to unseen words by combining familiar subunits.
  5. A drawback is that meaningful words may be split into too many pieces, increasing sequence length.
  6. Another issue is that some subwords are arbitrary fragments with little meaning.
  7. For English this tradeoff works well, but in highly agglutinative languages the splits may become excessive.

## Q4. Edit Distance
Find minimum edit distance between 'Sunday' and 'Saturday'.

In [24]:

def edit_distance(s1, s2, sub_cost=1, ins_cost=1, del_cost=1):
    m, n = len(s1), len(s2)
    dp = [[0]*(n+1) for _ in range(m+1)]

    for i in range(m+1):
        dp[i][0] = i * del_cost
    for j in range(n+1):
        dp[0][j] = j * ins_cost

    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = sub_cost
            dp[i][j] = min(
                dp[i-1][j-1] + cost,
                dp[i-1][j] + del_cost,
                dp[i][j-1] + ins_cost
            )
    return dp[m][n]

print("Model A (Sub=1, Ins=1, Del=1):", edit_distance("Sunday", "Saturday", 1,1,1))
print("Model B (Sub=2, Ins=1, Del=1):", edit_distance("Sunday", "Saturday", 2,1,1))

Model A (Sub=1, Ins=1, Del=1): 3
Model B (Sub=2, Ins=1, Del=1): 4


**Q4. Reflection:**

         1. Model A result: Distance = 3.
         
         2. Model B result: Distance = 4.
         
         3. The difference comes from the fact that substitution is cheaper in Model A, so the algorithm takes fewer steps. 
         
         4. In Model B, substitution is costly, leading to more insertions and deletions. 
         
         5. This is important in real-world applications: 
                 o	Spell check systems should use cheap substitutions since typos often replace one character with another. 
                 o	DNA and protein sequence alignment should favor cheaper insertions and deletions because mutations typically involve insertions or deletions rather than substitutions.